# Demo CoreEngine

In [1]:
from src.StreamPort.core.CoreEngine import CoreEngine

# Creates an empty CoreEngine object and prints it
x = CoreEngine()
x.print()

CoreEngine 
  name: None 
  author: None 
  path: None 
  date: 2024-03-23 13:03:43.772916 



## ProjectHeaders

In [1]:
from src.StreamPort.core.CoreEngine import CoreEngine
from src.StreamPort.core.ProjectHeaders import ProjectHeaders

x = CoreEngine()

print("1:")
# Creates a ProjectHeaders object and prints it 
ph = ProjectHeaders()
ph.print()

# Validates the ProjectHeaders object
print(ph.validate())

print("\n2:")
# Modifies headers in the CoreEngine
x.add_headers(headers={"name": "Example Name", "description": "demo description"})
x.print()

print("3:")
# Removes the description header
x.remove_headers("description")
print(x.get_headers())

1:

ProjectHeaders
  name: None
  author: None
  path: None
  date: 2024-03-23 14:08:15.835097

True

2:

CoreEngine 
  name: Example Name 
  author: None 
  path: None 
  date: 2024-03-23 14:08:15.835097 

3:
{'name': 'Example Name', 'author': None, 'path': None, 'date': datetime.datetime(2024, 3, 23, 14, 8, 15, 835097)}


## Analyses

In [2]:
from src.StreamPort.core.CoreEngine import CoreEngine
from src.StreamPort.core.Analyses import Analyses

x = CoreEngine()

print("1:")
# Creates an Analyses object and prints it
ana1 = Analyses(name = "Analysis 1")
ana1.print()

print("2:")
# An Analysis list
anaList = [ana1, Analyses(name = "Analysis 2"), Analyses(name = "Analysis 3")]
print("Number of analyses: ", anaList.__len__())

print("3:")
# Adds the list of analyses to the CoreEngine
x.add_analyses(anaList)
print("Number of analyses: ", x._analyses.__len__())

print("4:")
# Removes the first analysis from the list
x.remove_analyses(0)
print("Number of analyses: ", x._analyses.__len__())

print("5:")
# Removes all analyses from the list
x.remove_analyses()
print("Number of analyses: ", x._analyses.__len__())


1:

Analysis
  name: Analysis 1
  replicate: None
  blank: None

2:
Number of analyses:  3
3:
Number of analyses:  3
4:
Number of analyses:  2
5:
Number of analyses:  0


## ProcessingSettings

In [1]:
from src.StreamPort.core.CoreEngine import CoreEngine
from src.StreamPort.core.ProcessingSettings import ProcessingSettings

x = CoreEngine()

print("1:")
# Creates a ProcessingSettings object and prints it
settings = ProcessingSettings()
settings.print()

print("2:")
# Adds the settings to the CoreEngine
x.add_settings(settings)
print("Number of settings: ", x._settings.__len__())

print("3:")
# Removes the settings from the CoreEngine
x.remove_settings()
print("Number of settings: ", x._settings.__len__())


1:

 ProcessingSettings
 call         None
 algorithm    None
 version      None
 software     None
 developer    None
 contact      None
 link         None
 doi          None

 parameters: empty 

2:
Number of settings:  1
3:
Number of settings:  0


## Results

In [4]:
from src.StreamPort.core.CoreEngine import CoreEngine

x = CoreEngine()

print("1:")
# Creates a results dictionary with data from two sensors
res1 = {"sensors": {"sensor1": {"data": [1, 2, 3, 4, 5], "time": [1, 2, 3, 4, 5]}, "sensor2": {"data": [1, 2, 3, 4, 5], "time": [1, 2, 3, 4, 5]}}}
print(res1)

print("2:")
# Adds the results to the CoreEngine
x.add_results(res1)
print("Number of results: ", x._results.__len__())

print("3:")
# Removes the results sensors from the CoreEngine
x.remove_results("sensors")
print("Number of results: ", x._results.__len__())

1:
{'sensors': {'sensor1': {'data': [1, 2, 3, 4, 5], 'time': [1, 2, 3, 4, 5]}, 'sensor2': {'data': [1, 2, 3, 4, 5], 'time': [1, 2, 3, 4, 5]}}}
2:
Number of results:  1
3:
Number of results:  0


# Class Inheritance

Engines are created based on the class inheritance as described below.
Basic methods are reused from the core engine. 

In [ ]:
class Person:
  """Class docstrings go here."""

  def __init__(self, fname, lname):
    self.firstname = fname
    self.lastname = lname

  def printname(self):
    """Class method docstrings go here."""
    print(self.firstname, self.lastname)

  def welcome_from_parent(self):
    print("Welcome from parent class")

#Use the Person class to create an object, and then execute the printname method:

x = Person("John", "Doe")

x.printname()

class Student(Person):
  def __init__(self, fname, lname, year):
    super().__init__(fname, lname)
    self.graduationyear = year

  def welcome(self):
    print("Welcome", self.firstname, self.lastname, "to the class of", self.graduationyear)

  def printname(self):
    print(self.firstname, self.lastname, "overwritten!")

  def printname_from_parent(self):
    super().printname()

x = Student("Mike", "Olsen", 2019)

x.printname()

x.printname_from_parent()

x.welcome()

x.welcome_from_parent()

help(Person)

# Other Code Lines

In [ ]:
import sys
print(sys.executable)